In [1]:
import pandas as pd
import psycopg2
import numpy as np # linear algebra
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots
import datetime as dt

In [2]:
# Store environment variable
from getpass import getpass
dbPassword = getpass('Enter database password')

Enter database password··········


In [3]:
param_dic = {
        'database': 'big-data-bowl',
        'user': 'postgres',
        'password': dbPassword,
        'host': '34.72.136.99',
        'port': 5432,
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [5]:
# Connect to the database
conn = connect(param_dic)
column_names = ["gameId", "playId", "nflId", "week", "homeTeamAbbr", "visitorTeamAbbr",
                "height", "weight", "birthDate", "collegeName", "officialPosition", "displayName",
                "quarter", "down", "yardsToGo", "possessionTeam", "defensiveTeam", "yardlineNumber",
                "gameClock", "preSnapHomeScore", "preSnapVisitorScore", "playResult", "offensiveFormation",
                "personnelO", "defendersInTheBox", "personnelD", "dropbackType", "pff_playAction", "pff_passCoverage", "pff_passCoverageType",
                "pff_role", "pff_positionLinedUp", "pff_hit", "pff_hurry", "pff_sack", "pff_beatenByDefender",
                "pff_hitAllowed", "pff_hurryAllowed", "pff_sackAllowed", "pff_nflIdBlockedPlayer", "pff_blockType", "pff_backFieldBlock",
                ]
query = "SELECT pff.\"gameId\", pff.\"playId\",pff.\"nflId\",       \
     gm.week, gm.\"homeTeamAbbr\", gm.\"visitorTeamAbbr\",        \
	   plr.height, plr.weight, plr.\"birthDate\",                     \
	   plr.\"collegeName\", plr.\"officialPosition\",                 \
	   plr.\"displayName\",                                           \
	   pl.quarter, pl.down, pl.\"yardsToGo\", pl.\"possessionTeam\",  \
	   pl.\"defensiveTeam\", pl.\"yardlineNumber\", pl.\"gameClock\", \
	   pl.\"preSnapHomeScore\", pl.\"preSnapVisitorScore\",           \
	   pl.\"playResult\", pl.\"offenseFormation\",                    \
	   pl.\"personnelO\", pl.\"defendersInBox\", pl.\"personnelD\",   \
	   pl.\"dropbackType\",  pl.\"pff_playAction\",                   \
	   pl.\"pff_passCoverage\", pl.\"pff_passCoverageType\",          \
	   pff.pff_role, pff.\"pff_positionLinedUp\", pff.pff_hit,        \
	   pff.pff_hurry, pff.pff_sack, pff.\"pff_beatenByDefender\",     \
	   pff.\"pff_hitAllowed\", pff.\"pff_hurryAllowed\",              \
	   pff.\"pff_sackAllowed\", pff.\"pff_nflIdBlockedPlayer\",       \
	   pff.\"pff_blockType\", pff.\"pff_backFieldBlock\"              \
FROM pffscoutingdata as pff                                         \
LEFT JOIN plays as pl                                               \
ON pff.\"gameId\" = pl.\"gameId\"                                   \
AND pff.\"playId\" = pl.\"playId\"                                  \
LEFT JOIN games as gm                                               \
ON pff.\"gameId\" = gm.\"gameId\"                                   \
LEFT JOIN players as plr                                            \
ON pff.\"nflId\" = plr.\"nflID\"                                    \
WHERE plr.\"officialPosition\" IN ('C', 'G', 'T', 'TE') "

# Execute the "SELECT" query
pff_joined_df = postgresql_to_dataframe(conn, query, column_names)
pff_joined_df.head()

Connecting to the PostgreSQL database...
Connection successful


,gameId,playId,nflId,week,homeTeamAbbr,visitorTeamAbbr,height,weight,birthDate,collegeName,...,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021101400,3278,44831,6,PHI,TB,6-6,250,1994-11-18,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
1,2021090900,97,35481,1,TB,DAL,6-6,265,1989-05-14,Arizona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
2,2021101400,3278,46163,6,PHI,TB,6-6,305,1995-01-27,Humboldt State,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NB,0.0
3,2021101400,3278,52421,6,PHI,TB,6-5,322,1999-01-24,Iowa,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,46199.0,PP,0.0
4,2021090900,97,40151,1,TB,DAL,6-4,319,1991-05-27,Colorado State-Pueblo,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0


Create new player variables

In [6]:
from operator import index
players = pff_joined_df[['nflId','birthDate','weight','height','pff_positionLinedUp','pff_blockType']]
players = players.set_index('nflId')
players['birthDate'].replace('NA','', inplace=True)
players['birthDate'] = pd.to_datetime(players['birthDate'])
players['age'] = players['birthDate'].map(lambda x: dt.date.today().year-x.year)
players['age'] = players['age'].fillna(0)
players = players.drop(columns='birthDate')

# players['birthMonth'] = players['birthDate'].map(lambda x: x.month)
# players['birthYear'] = players['birthDate'].map(lambda x: x.year)

players['heightCm'] = players['height'].map(lambda x: int(x.split('-')[0])*30.48+int(x.split('-')[1])*2.54)
players = players.drop(columns='height')

print('dataframe size: {}'.format(players.shape))

players.head()

dataframe size: (53382, 5)


,weight,pff_positionLinedUp,pff_blockType,age,heightCm
nflId,,,,,
44831,250,TE-iR,,28.0,198.12
35481,265,TE-L,,33.0,198.12
46163,305,RG,NB,27.0,198.12
52421,322,RT,PP,23.0,195.58
40151,319,C,SW,31.0,193.04


In [7]:
players.dtypes

weight                   int64
pff_positionLinedUp     object
pff_blockType           object
age                    float64
heightCm               float64
dtype: object

PART 1 - Simple Analysis

About 75% of the players are under 32 years old

The youngest players are 23 years old, and there are 32 of them

The oldest player is 45 years old

In [8]:
round(players.age.value_counts(normalize=True, bins=5)*100, 2)

(24.6, 32.8]     76.57
(-0.042, 8.2]     9.81
(32.8, 41.0]      9.13
(16.4, 24.6]      4.48
(8.2, 16.4]       0.00
Name: age, dtype: float64

In [9]:
fig = px.density_heatmap(players, x='heightCm', y='weight', title='Player height and weight distribution')
fig.show()

In [10]:
fig = px.scatter(players, x='heightCm', y='weight', trendline="ols")
fig.show()

In [11]:
players_by_age = players.groupby('age').count().reset_index()
players_by_age = players_by_age.rename(columns={"heightCm" : "count"})
fig = px.bar(players_by_age, x='age', y='count', title="Count of Players by Ages")
fig.show()

In [12]:
fig = px.scatter_3d(
    players, x='age', y='heightCm', z='weight',
    color='pff_positionLinedUp', opacity=0.7
)
fig.update_layout(height=800, title='Player stats of different positions')
fig.show()

Part 2: Performance Analysis

In [13]:
players["inTrouble"] = pff_joined_df[['pff_hitAllowed','pff_hurryAllowed','pff_sackAllowed','pff_beatenByDefender']].any(axis='columns')
players.head()

,weight,pff_positionLinedUp,pff_blockType,age,heightCm,inTrouble
nflId,,,,,,
44831,250,TE-iR,,28.0,198.12,False
35481,265,TE-L,,33.0,198.12,False
46163,305,RG,NB,27.0,198.12,False
52421,322,RT,PP,23.0,195.58,False
40151,319,C,SW,31.0,193.04,False


In [14]:
players_encoded = players.copy()
# Binary encoding using Pandas (multiple columns)
players_encoded = pd.get_dummies(players, columns=['pff_positionLinedUp','pff_blockType','inTrouble'])
players_encoded = players_encoded.drop(columns='inTrouble_False')
players_encoded.head()

,weight,age,heightCm,pff_positionLinedUp_C,pff_positionLinedUp_DRT,pff_positionLinedUp_FB,pff_positionLinedUp_FB-L,pff_positionLinedUp_FB-R,pff_positionLinedUp_HB-L,pff_positionLinedUp_HB-R,...,pff_blockType_NB,pff_blockType_PA,pff_blockType_PP,pff_blockType_PR,pff_blockType_PT,pff_blockType_PU,pff_blockType_SR,pff_blockType_SW,pff_blockType_UP,inTrouble_True
nflId,,,,,,,,,,,,,,,,,,,,,
44831,250,28.0,198.12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35481,265,33.0,198.12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46163,305,27.0,198.12,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
52421,322,23.0,195.58,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
40151,319,31.0,193.04,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
players_encoded.isnull().sum()

weight                       0
age                          0
heightCm                     0
pff_positionLinedUp_C        0
pff_positionLinedUp_DRT      0
pff_positionLinedUp_FB       0
pff_positionLinedUp_FB-L     0
pff_positionLinedUp_FB-R     0
pff_positionLinedUp_HB-L     0
pff_positionLinedUp_HB-R     0
pff_positionLinedUp_LG       0
pff_positionLinedUp_LT       0
pff_positionLinedUp_LWR      0
pff_positionLinedUp_NLT      0
pff_positionLinedUp_NRT      0
pff_positionLinedUp_RG       0
pff_positionLinedUp_RT       0
pff_positionLinedUp_RWR      0
pff_positionLinedUp_SLWR     0
pff_positionLinedUp_SLiWR    0
pff_positionLinedUp_SLoWR    0
pff_positionLinedUp_SRWR     0
pff_positionLinedUp_SRiWR    0
pff_positionLinedUp_SRoWR    0
pff_positionLinedUp_TE-L     0
pff_positionLinedUp_TE-R     0
pff_positionLinedUp_TE-iL    0
pff_positionLinedUp_TE-iR    0
pff_positionLinedUp_TE-oL    0
pff_positionLinedUp_TE-oR    0
pff_blockType_               0
pff_blockType_BH             0
pff_bloc

In [16]:
# Create a training set 'X' with every column except 'inTrouble_True'
X = players_encoded.drop('inTrouble_True', axis=1)
y = players_encoded['inTrouble_True']

# Scale the dataset using MinMaxScaler()
from sklearn.preprocessing import MinMaxScaler
X_scaled = MinMaxScaler().fit_transform(X)
X_scaled

array([[0.23076923, 0.68292683, 0.66666667, ..., 0.        , 0.        ,
        0.        ],
       [0.31952663, 0.80487805, 0.66666667, ..., 0.        , 0.        ,
        0.        ],
       [0.55621302, 0.65853659, 0.66666667, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.20118343, 0.75609756, 0.55555556, ..., 0.        , 0.        ,
        0.        ],
       [0.75147929, 0.70731707, 0.66666667, ..., 0.        , 0.        ,
        0.        ],
       [0.56804734, 0.70731707, 0.44444444, ..., 0.        , 0.        ,
        0.        ]])

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [19]:
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [20]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.913677690078929
Testing Data Score: 0.9133822868275139
